In [1]:
import sys
sys.path.insert(1, '../FeatureEngineering')

import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import scipy
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
from util import plot_history
import feature_builder

Cargo datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Armo features

In [3]:
X_train = feature_builder.process_dataset(train_df)
y_train = train_df['target'].values

Percentage of words covered in the embeddings = 0.6336399642263958


In [4]:
X_train.columns

Index(['invalid_location_character_count', 'location_is_place', 'mean_encode',
       'keyword_length', 'text_length', 'location_length', 'stop_word_count',
       'punctuation_count', 'hashtag_count', 'mention_count',
       ...
       'text_embedding_290', 'text_embedding_291', 'text_embedding_292',
       'text_embedding_293', 'text_embedding_294', 'text_embedding_295',
       'text_embedding_296', 'text_embedding_297', 'text_embedding_298',
       'text_embedding_299'],
      dtype='object', length=334)

In [14]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    model.summary()
    return model

model = create_model()
history = model.fit(X_train, y_train, epochs=35, shuffle=False, validation_split=0.0)
plot_history(history)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               42880     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 51,201
Trainable params: 51,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/35
238/238 [==============================] - 0s 1ms/step - loss: 1.5198 - accuracy: 0.5517
Epoch 2/35
238/238 [==============================] - 0s 1ms/step - loss: 0.6590 - accuracy: 0.6359
Epoch 3/35
238/238 [==============================] - 0s 1ms/step - loss: 0.6271 - accuracy: 0.6679
Epoch 4/35
238/238 [==============================] - 0s 1ms/step - loss: 0.6022

KeyError: 'val_accuracy'

In [15]:
test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(test)
pd.Series([x[0] for x in prediction]).value_counts()

Percentage of words covered in the embeddings = 0.6973331905322909


0    2073
1    1190
dtype: int64

Calculemos el test para kaggle

In [16]:
ids = test_df['id']

final_test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(final_test)
final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

Percentage of words covered in the embeddings = 0.6973331905322909


,target
id,
0,1
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [17]:
final_df['target'].value_counts()

0    2073
1    1190
Name: target, dtype: int64

In [18]:
final_df.count()

target    3263
dtype: int64

In [19]:
final_df.to_csv('nn.csv')